In [ ]:
import cv2
import numpy as np

# Funkcja do skalowania obrazu za pomocą filtru uśredniającego
def skaluj_obraz(obraz, wspolczynnik_skalowania):
    rozmiar_jadra = int(1 / wspolczynnik_skalowania) * 2 + 1
    jadro = np.ones((rozmiar_jadra, rozmiar_jadra), np.float32) / (rozmiar_jadra ** 2)
    obraz_skalowany = cv2.filter2D(obraz, -1, jadro)
    return obraz_skalowany

# Funkcja interpolacji sąsiada
def interpolacja_sasiada(obraz, wspolczynnik_skalowania):
    wysokosc, szerokosc, _ = obraz.shape
    nowa_wysokosc = int(wysokosc * wspolczynnik_skalowania)
    nowa_szerokosc = int(szerokosc * wspolczynnik_skalowania)
    obraz_skalowany = np.zeros((nowa_wysokosc, nowa_szerokosc, 3), dtype=np.uint8)

    for i in range(nowa_wysokosc):
        for j in range(nowa_szerokosc):
            x = int(j / wspolczynnik_skalowania)
            y = int(i / wspolczynnik_skalowania)
            obraz_skalowany[i, j] = obraz[y, x]

    return obraz_skalowany

# Funkcja interpolacji dwuliniowej
def interpolacja_dwuliniowa(obraz, k_skalowania):
    wysokosc, szerokosc, _ = obraz.shape
    nowa_wysokosc = int(wysokosc * k_skalowania)
    nowa_szerokosc = int(szerokosc * k_skalowania)
    obraz_skalowany = np.zeros((nowa_wysokosc, nowa_szerokosc, 3), dtype=np.uint8)

    for i in range(nowa_wysokosc):
        for j in range(nowa_szerokosc):
            x = j / k_skalowania
            y = i / k_skalowania
            x0, y0 = int(x), int(y)
            x1, y1 = min(x0 + 1, szerokosc - 1), min(y0 + 1, wysokosc - 1)
            fx, fy = x - x0, y - y0
            for c in range(3):
                interp_wartosc = (1 - fx) * (1 - fy) * obraz[y0, x0, c] + \
                                 fx * (1 - fy) * obraz[y0, x1, c] + \
                                 (1 - fx) * fy * obraz[y1, x0, c] + \
                                 fx * fy * obraz[y1, x1, c]
                obraz_skalowany[i, j, c] = int(interp_wartosc)

    return obraz_skalowany

# Funkcja max pooling
def max_pooling(obraz, rozmiar_poolingu):
    wysokosc, szerokosc, _ = obraz.shape
    nowa_wysokosc = wysokosc // rozmiar_poolingu
    nowa_szerokosc = szerokosc // rozmiar_poolingu
    obraz_poolowany = np.zeros((nowa_wysokosc, nowa_szerokosc, 3), dtype=np.uint8)

    for i in range(nowa_wysokosc):
        for j in range(nowa_szerokosc):
            wierszs, wierszk = i * rozmiar_poolingu, (i + 1) * rozmiar_poolingu
            kolumnas, kolumnak = j * rozmiar_poolingu, (j + 1) * rozmiar_poolingu
            region = obraz[wierszs:wierszk, kolumnas:kolumnak, :]
            obraz_poolowany[i, j, :] = np.max(region, axis=(0, 1))

    return obraz_poolowany

# Funkcja skalująca obraz w zależności od współczynnika skalowania
def skaluj_obraz1(obraz, wspolczynnik_skalowania):
    if wspolczynnik_skalowania > 1:
        return interpolacja_dwuliniowa(obraz, wspolczynnik_skalowania)
    else:
        rozmiar_poolingu = int(1 / wspolczynnik_skalowania)
        return max_pooling(obraz, rozmiar_poolingu)

# Funkcja dokonująca operacji sekwencyjnie w zależności od współczynnika skalowania
def sekwencyjnie(obraz, wspolczynnik_skalowania):
    if wspolczynnik_skalowania > 1:
        return interpolacja_dwuliniowa(obraz, wspolczynnik_skalowania)
    elif wspolczynnik_skalowania < 1:
        rozmiar_poolingu = int(1 / wspolczynnik_skalowania)
        return max_pooling(obraz, rozmiar_poolingu)
    else:
        return obraz

# Funkcja obliczająca błąd średniokwadratowy (MSE) pomiędzy dwoma obrazami
def mse(oryginal, przerobiony):
    min_wysokosc = min(oryginal.shape[0], przerobiony.shape[0])
    min_szerokosc = min(oryginal.shape[1], przerobiony.shape[1])
    oryginal = oryginal[:min_wysokosc, :min_szerokosc, :]
    przerobiony = przerobiony[:min_wysokosc, :min_szerokosc, :]
    return np.mean((oryginal - przerobiony) ** 2)

# Wczytanie oryginalnego obrazu
oryginalny_obraz = cv2.imread('C:/Users/PPiwnicki/Downloads/zdj.jpg')
wspolczynnik_skalowania = 0.5

# Skalowanie obrazu za pomocą różnych metod
obraz_skalowany = skaluj_obraz(oryginalny_obraz, wspolczynnik_skalowania)
obraz_skalowany1 = skaluj_obraz1(oryginalny_obraz, wspolczynnik_skalowania)

# Wykonanie operacji bezpośrednio i sekwencyjnie
